References: https://docs.pinecone.io/docs/

Format of vector data:

 vectors=[
    {
      "id": "A",
      "values": [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
      "metadata": {"genre": "comedy", "year": 2020}
    },
    {
      "id": "B",
      "values": [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
      "metadata": {"genre": "documentary", "year": 2019}
    },
    {
      "id": "C",
      "values": [0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3, 0.3],
      "metadata": {"genre": "comedy", "year": 2019}
    },
    {
      "id": "D",
      "values": [0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4, 0.4],
      "metadata": {"genre": "drama"}
    }
  ]
)


In [1]:
# !pip install pinecone-client
# !pip install langchain
#!pip install -U langchain-openai

In [2]:
from langchain_openai import OpenAIEmbeddings
from langchain.schema import Document


In [3]:
import pandas as pd
from tqdm import tqdm
import os, time
from itertools import islice


In [4]:
ENVIRONMENT = 'gcp-starter'

In [5]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key= PINE_API_KEY, environment = ENVIRONMENT)


In [6]:
pc

In [7]:
myindex = pc.Index(host=os.environ(HOSTNAME))


In [8]:
#OpenAI Config
token = os.environ(API_KEY)

model_name = 'text-embedding-ada-002'
embed = OpenAIEmbeddings(model=model_name,openai_api_key=token)

In [9]:
df = pd.read_csv('/content/recipes_finalv1.csv', on_bad_lines='skip')
df = df.fillna('')
df=df[:1000]

<ipython-input-9-9a898559b235>:1: DtypeWarning: Columns (9,17,21,22,23,24) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/recipes_finalv1.csv', on_bad_lines='skip')


In [10]:
df.rename(columns={ df.columns[0]: "ID" },inplace=True)

In [11]:
df.columns

Index(['ID', 'CookTime', 'PrepTime', 'TotalTime', 'Description',
       'RecipeCategory', 'Keywords', 'AggregatedRating', 'Calories',
       'FatContent', 'SaturatedFatContent', 'CholesterolContent',
       'SodiumContent', 'CarbohydrateContent', 'FiberContent', 'SugarContent',
       'ProteinContent', 'RecipeServings', 'RecipeYield', 'RecipeInstructions',
       'Ingredients', 'ProteinPercentage', 'CarbohydratePercentage',
       'SugarPercentage', 'FatPercentage'],
      dtype='object')

In [12]:
myindex.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

In [13]:
# Get description vectors and metadata
veci = df['Description']
mdata = df.drop(columns=['Description','ID'])
id = df['ID']

In [14]:
texts_to_embed = veci.tolist()

In [15]:
embeddings = embed.embed_documents(texts_to_embed)

In [16]:
len(embeddings)

1000

In [17]:
dict_data = mdata.to_dict(orient='records')
print(dict_data)

[{'CookTime': '0:20:00', 'PrepTime': '0:15:00', 'TotalTime': '0:35:00', 'RecipeCategory': 'Lunch/Snacks', 'Keywords': "['Chicken', 'Poultry', 'Meat', 'Asian', 'Kosher', '< 60 Mins', 'Stove Top', 'Stir Fry']", 'AggregatedRating': 4.0, 'Calories': 212.1, 'FatContent': '10.9', 'SaturatedFatContent': 1.1, 'CholesterolContent': 54.9, 'SodiumContent': 191.6, 'CarbohydrateContent': 3.9, 'FiberContent': 1.4, 'SugarContent': 1.7, 'ProteinContent': 23.8, 'RecipeServings': 6.0, 'RecipeYield': '', 'RecipeInstructions': 'Snip tips off snow peas.\nWash them and set aside.\nCut chicken into 1/4-inch wide strips, each about 3-inches long.\nToss chicken with coriander in a small bowl.\nIn a nonstick wok or heavy skillet, heat half the oil over medium-high heat.\nAdd half the chicken and stir-fry just until cooked, about 2 minutes.\nUse a slotted spoon to remove chicken to a plate.\nHeat remaining oil and stir-fry the rest of the chicken until it is cooked.\nRemove to a plate, using slotted spoon.", \n"

In [29]:
for i in range(len(id)):
    myindex.upsert(vectors=[{"id": id[i], "values": embeddings[i], "metadata": dict_data[i]}])
print("Successfully updated to db!")

Successfully updated to db!


In [30]:
#########################################################################################################################################